In [1]:
import breakwater as bw
import pandas as pd
import os
from pathlib import Path
import numpy as np

In [2]:
from development_overtopping_DKA import eurotop2018_6_5, surf_similarity, gamma_beta_eurotop_2018_6_9


In [5]:
project_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx",
                             index_col = 1,
                             sheet_name='Input_Project specific')
requirements_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx",
                                  index_col = 0,
                                  sheet_name='Input_requirements')
wave_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx",
                          index_col = 0,
                          sheet_name='input_hydrotechnical',
                         skiprows = 1)
cross_section_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx", 
                                   sheet_name='Input_Cross section',
                                  skiprows = 1)

In [6]:
project_data

,Parameter,Unit,Explanation,Value
Symbol,,,,
rho_w,Water density,kg/m3,NaN,1010
rho_r,Rock density,kg/m3,NaN,2600
rho_c,Concrete density,kg/m3,NaN,2350
g,Gravity,m/s^2,NaN,9.81
ELS,Exploitation limit state,-,NaN,ELS
SLS,Servicability limit state,-,NaN,SLS
ULS,Ultimate limit state,-,NaN,ULS
Cpl_deep,Plunging coefficient deep,-,NaN,6.2
Cs_deep,Surging coefficient deep,-,NaN,1


In [7]:
requirements_data

,Symbol,Unit,Explanation,ELS,SLS,ULS
Parameter,,,,,,
Overtopping limit public access,q_max,l/m/s,NaN,0.1,5,200
Overtopping limit restricted access,q_max,l/m/s,NaN,50,-,-
Armour damage limit,Sd_max,-,NaN,-,2,8
Toe damage limit,Nod_max_toe,-,NaN,-,0.5,4
Concrete armour damage limit,Nod_max_armour,-,NaN,-,0,0.5


In [8]:
wave_data

,Structure,Chainage,Offshore bin,Limit State,wl,Hm0,dir_wave,Tp,"Tm-1,0","Tm0,2"
Calculation_case,,,,,,,,,,
1,ECn2,370N,0,ELS,0.38,1.060,48.1,5.58,3.95,2.78
2,ECn2,370N,30,ELS,0.47,2.060,69.1,8.11,6.66,4.85
3,ECn2,370N,60,ELS,0.56,2.800,78.9,9.01,8.05,6.60
4,ECn2,370N,90,ELS,0.50,2.570,86.5,9.00,7.85,6.30
5,ECn2,370N,120,ELS,0.39,1.650,98.0,7.07,6.02,4.77
...,...,...,...,...,...,...,...,...,...,...
164,ECn2,120N,90,ULS,0.85,1.908,72.7,12.03,11.73,10.95
165,ECn2,120N,120,ULS,0.63,1.584,74.7,9.89,9.49,8.72
166,ECn2,120N,150,ULS,0.65,1.464,78.1,9.62,8.98,7.99


In [9]:
cross_section_data


,Structure,Chainage,dir_structure,z_bed,public,P,tan_a,Gc,slope_foreshore,gamma_f_c,cot_a_rear,B,roundhead,safety
0,ECn2,370N,353.4,-6.61,0,0.5,0.666667,1.5,300,0.55,1.5,6,1,0
1,ECn2,320N,353.4,-6.52,0,0.5,0.666667,1.5,300,0.55,1.5,6,0,0
2,ECn2,270N,353.4,-4.41,0,0.5,0.666667,1.5,300,0.55,1.5,6,0,0
3,ECn2,220N,353.4,-3.32,0,0.5,0.666667,1.5,300,0.55,1.5,6,0,0
4,ECn2,190N,353.4,-3.00,0,0.5,0.666667,1.5,300,0.55,1.5,6,0,0
5,ECn2,170N,353.4,-2.29,1,0.5,0.666667,1.5,300,0.55,1.5,6,0,0
6,ECn2,120N,353.4,-1.21,1,0.5,0.666667,1.5,300,0.55,1.5,6,0,0


In [17]:
# CALCULATE CREST HEIGHT FOR OVERTOPPING

# Do we want to put this entire part in a function to keep the scripts clean a bit?

# Still necessary: Loop over all calculation cases for a cross section

# Still necessary: Error/notification if there is no overtopping limit

# Still necessary: implementation to select correct overtopping limit depending
# on if public access is allowed

# Still necessary: implementation to select correct roughness
# depending on armour case.

Calculation_case = 3
Cross_section_id = 0
# Public access: allowed
# Armour = 2 layers rock, permeable core

# Open project specific parameters
g = project_data.at['g', 'Value']

# Open structure specific parameters
tana          = cross_section_data.at[Cross_section_id, 'tan_a']
dir_structure = cross_section_data.at[Cross_section_id, 'dir_structure']
safety        = cross_section_data.at[Cross_section_id, 'safety']

# Get info for sea state. Currently implemented to do only the first sea state
Hm0      = wave_data.at[Calculation_case, 'Hm0']
wl       = wave_data.at[Calculation_case, 'wl']
Tm_min_1 = wave_data.at[Calculation_case, 'Tm-1,0']
dir_wave = wave_data.at[Calculation_case, 'dir_wave']

# Get allowed q for the location and calculation case
LS        = wave_data.at[Calculation_case, 'Limit State']
q_allowed = requirements_data.at['Overtopping limit public access', LS]


#Calculate roughness reduction
# Set armour for rougness reduction
armour_layer = 'Rock'
layers       = 2
permeability = 'permeable'

# Calculate surf-similarity with Hm0 and Tm-1,0
xi_m_min_1 = surf_similarity(tana, Hm0, Tm_min_1, g)

gamma_f = bw.core.overtopping.gamma_f('Rock', xi_m_min_1, layers = '2', permeability = 'permeable')

#Calculate obliqueness reduction
beta       = dir_wave-(dir_structure-360) #Done like this for ECn2 north. Should find more robust solution
gamma_beta = gamma_beta_eurotop_2018_6_9(beta)


Rc = eurotop2018_6_5(g, Hm0, q_allowed, gamma_f, gamma_beta, limit = False, safety=safety)
z_crest = Rc + wl

In [19]:
#print("Cross-section : "+str(cross_section_data.at[Cross_section_id,"Structure"])+" - "+str(cross_section_data.at[Cross_section_id,"Chainage"]))
#print("Wave info for : " +str(wave_data.at[Calculation_case,"Structure"])+" - "+str(wave_data.at[Calculation_case,"Chainage"]))      
print("q_allowed : "+str(q_allowed))
#print("g         : "+str(g))
#print("Hm0       : "+str(Hm0))
#print("Tm_min_1  : "+str(Tm_min_1))
#print("wl        : "+str(wl))
print("xi_m_min_1: "+str(xi_m_min_1))
print("Rc        : "+str(Rc))
print("beta      : "+str(beta))
print("gamma_f   : "+str(gamma_f))
print("gamma_beta: "+str(gamma_beta))
print("z_crest   : "+str(z_crest))


q_allowed : 0.1
xi_m_min_1: 4.007469033422432
Rc        : 2.090738677277467
beta      : 85.50000000000003
gamma_f   : 0.4
gamma_beta: 0.496
z_crest   : 2.650738677277467


In [13]:
# CALCULATE REQUIRED STONE DIAMETER (Rock armour)

Calculation_case = 57
Cross_section_id = 0
# Armour = 2 layers rock, permeable core

Storm_duration = 6 #Hours
Safety = 0 #Safety factor on top of regular VDM constants

# Open project specific parameters
g     = project_data.at['g', 'Value']
rho_a = project_data.at['rho_r', 'Value'] #For rock armour. Needs adaptation for concrete
rho_w = project_data.at['rho_w', 'Value']
Cpl_shallow = project_data.at['Cpl_shallow', 'Value']
Cs_shallow = project_data.at['Cs_shallow', 'Value']

#Get info for sea state
Hm0      = wave_data.at[Calculation_case, 'Hm0']
wl       = wave_data.at[Calculation_case, 'wl']
Tm_min_1 = wave_data.at[Calculation_case, 'Tm-1,0']
dir_wave = wave_data.at[Calculation_case, 'dir_wave']
Tm_0_2   = wave_data.at[Calculation_case, 'Tm0,2']
N        = np.round(Storm_duration*3600/Tm_0_2)


# Open structure specific parameters
tana            = cross_section_data.at[Cross_section_id, 'tan_a']
dir_structure   = cross_section_data.at[Cross_section_id, 'dir_structure']
safety          = cross_section_data.at[Cross_section_id, 'safety']
z_bed           = cross_section_data.at[Cross_section_id, 'z_bed']
slope_foreshore = cross_section_data.at[Cross_section_id, 'slope_foreshore']
P               = cross_section_data.at[Cross_section_id, 'P']


# Get allowed Sd for the location and calculation case
LS         = wave_data.at[Calculation_case, 'Limit State']
Sd_allowed = requirements_data.at['Armour damage limit', LS]

#Intermediate calculations
h              = wl-z_bed
waveinfo       = bw.BattjesGroenendijk(Hm0, h, slope_foreshore)
H2_per         = waveinfo.get_Hp(0.02)
Hs             = waveinfo.get_Hn(3) #NU BATTJES-GROENENDIJK VOOR Hs UIT Hmo. IS DAT WAT WE WILLEN?

#OVERRIDE wave characteristics BECAUSE THERE IS A DIFFERENCE BETWEEN W+B SHEET AND BW TOOL
Hs = Hm0
H2_per = Hs*1.34
print("Warning: Hs and H2_per taken from W+B calculation")

Delta          = (rho_a-rho_w)/rho_w
xi_s_min_1     = surf_similarity(tana, Hs, Tm_0_2, g)
alpha          = np.arctan(tana)



# Check validity of Van der Meer shallow
vdm_shallow_validity = h/Hs

#Calculate required stone diameter without reduction
Dn50 = bw.core.vandermeer_shallow(Hs, 
                                  H2_per, 
                                  Delta, 
                                  P, 
                                  Sd_allowed, 
                                  N, 
                                  xi_s_min_1, 
                                  alpha, 
                                  Cpl = Cpl_shallow, 
                                  Cs = Cs_shallow, 
                                  safety = Safety)

#Reduce Dn50 with reduction factors from DAR


In [14]:
print("Cross-section : "+str(cross_section_data.at[Cross_section_id,"Structure"])+" - "+str(cross_section_data.at[Cross_section_id,"Chainage"]))
print("Wave info for : " +str(wave_data.at[Calculation_case,"Structure"])+" - "+str(wave_data.at[Calculation_case,"Chainage"]))      
print("Limit State   : "+str(LS))
print("Allowed Sd    : "+str(Sd_allowed))
print("g         : "+str(g))
print("h         : "+str(h))
print("Delta     : "+str(Delta))
print("P         : "+str(P))
print("Tangent a : "+str(tana))
print("Hs        : "+str(Hs))
print("H2%       : "+str(H2_per))
print("H2%/Hs    : "+str(H2_per/Hs))
print("Tm0,2     : "+str(Tm_0_2))
print("N         : "+str(N))
print("xi_s_min_1: "+str(xi_s_min_1))
print("Dn50      : "+str(Dn50))
print("h/Hs (<3) : "+str(vdm_shallow_validity))


Cross-section : ECn2 - 370N
Wave info for : ECn2 - 370N
Limit State   : SLS
Allowed Sd    : 2
g         : 9.81
h         : 7.1000000000000005
Delta     : 1.5742574257425743
P         : 0.5
Tangent a : 0.6666666666667
Hs        : 2.24
H2%       : 3.0016000000000003
H2%/Hs    : 1.34
Tm0,2     : 4.48
N         : 4821.0
xi_s_min_1: 2.493488188231773
Dn50      : 0.796993142080559
h/Hs (<3) : 3.169642857142857


In [ ]:
Hs/Hm0
